<a href="https://colab.research.google.com/github/DanielBR0612/unsupervised-network-ids/blob/main/Unsupervised_Network_Anomaly_Detection_CSE_CIC_2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importar o dataset

In [4]:
import os

meu_usuario = "danielbragaa"
minha_chave = "KGAT_7f527e1e30ae41a871547d40663449d7"

# Cria o diretorio
!mkdir -p ~/.kaggle

with open('/root/.kaggle/kaggle.json', 'w') as f:
    f.write(f'{{"username":"{meu_usuario}","key":"{minha_chave}"}}')

# 3. Muda a permissão do arquivo (exigência de segurança do Kaggle)
!chmod 600 ~/.kaggle/kaggle.json

# 4. Instala a biblioteca do Kaggle
!pip install -q kaggle

print("Configuração concluída")

# Baixar o dataset específico
print("Iniciando download...")
!kaggle datasets download -d solarmainframe/ids-intrusion-csv

# Descompactar o arquivo zip
print("Descompactando arquivos...")
!unzip -q ids-intrusion-csv.zip


# Ao carregar o dataset, foi baixado vários arquivos que correspondem a vários tipos de ataque, listei aqui para vc poder selecionar qual quer analisar

arquivos_disponiveis = {
    # ATAQUES DE FORÇA BRUTA & ACESSO
    '02-14-2018.csv': 'FTP-BruteForce, SSH-Bruteforce',

    # ATAQUES DE NEGAÇÃO DE SERVIÇO (DoS)
    '02-15-2018.csv': 'DoS-GoldenEye, DoS-Slowloris',
    '02-16-2018.csv': 'DoS-SlowHTTPTest, DoS-Hulk',

    # ATAQUES DISTRIBUÍDOS (DDoS)
    '02-20-2018.csv': 'DDoS-LOIC-HTTP, DDoS-LOIC-UDP',
    '02-21-2018.csv': 'DDoS-LOIC-UDP, DDoS-HOIC',
    # ATAQUES WEB (SQL INJECTION / XSS)
    '02-22-2018.csv': 'Web Brute Force, XSS, SQL Inj',
    '02-23-2018.csv': 'Web Attack - SQL Injection',

    # OUTROS
    '02-28-2018.csv': 'Infiltration', # evitar esse por enquanto, dificil de analisar
    '03-01-2018.csv': 'Botnet',
    '03-02-2018.csv': 'Botnet'
}


Configuração concluída
Iniciando download...
Dataset URL: https://www.kaggle.com/datasets/solarmainframe/ids-intrusion-csv
License(s): Attribution 4.0 International (CC BY 4.0)
ids-intrusion-csv.zip: Skipping, found more recently modified local copy (use --force to force download)
Descompactando arquivos...
replace 02-14-2018.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Juntar vários arquivos em apenas um

In [5]:
import pandas as pd
import numpy as np
import gc

arquivos_para_analise = [
    '02-14-2018.csv',
    '02-16-2018.csv'
]

FRACAO_AMOSTRA = 0.10

lista_dfs = []

print(f"Iniciando carga de {len(arquivos_para_analise)} arquivos...")

for arquivo in arquivos_para_analise:
    print(f"--> Lendo {arquivo}...")

    df_temp = pd.read_csv(arquivo, low_memory=False)

    df_sample = df_temp.sample(frac=FRACAO_AMOSTRA, random_state=42)

    lista_dfs.append(df_sample)

    del df_temp
    gc.collect()

print("Combinando os datasets...")
df_final = pd.concat(lista_dfs, axis=0)

df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)

print("="*50)
print(f"DATASET FINAL CRIADO!")
print(f"Dimensões Totais: {df_final.shape}")
print("="*50)

Iniciando carga de 2 arquivos...
--> Lendo 02-14-2018.csv...
    Tamanho original: 1048575 | Amostra: 104858
    Distribuição de Labels na amostra:
Label
Benign            66412
FTP-BruteForce    19458
SSH-Bruteforce    18988
Name: count, dtype: int64

--> Lendo 02-16-2018.csv...
    Tamanho original: 1048575 | Amostra: 104858
    Distribuição de Labels na amostra:
Label
DoS attacks-Hulk            46162
Benign                      44674
DoS attacks-SlowHTTPTest    14022
Name: count, dtype: int64

Combinando os datasets...
DATASET FINAL CRIADO!
Dimensões Totais: (209716, 80)


# Limpeza dos dados

In [7]:
colunas_para_remover = [
    'Label',
    'Timestamp',
    'Flow ID',
    'Src IP', 'Dst IP',
    'Src Port'
]

X = df_final.drop(columns=colunas_para_remover, errors='ignore')

print(f"Dados para o Modelo (X): {X.shape}")

Dados para o Modelo (X): (209716, 78) -> O modelo vai ver SÓ isso.

Verificando tipos de colunas restantes em X:
['Dst Port', 'Protocol', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg', 'Fw

In [8]:
import numpy as np

print(f"Tamanho antes: {X.shape}")

X = X.apply(pd.to_numeric, errors='coerce')

X.replace([np.inf, -np.inf], np.nan, inplace=True)

indices_validos = X.dropna().index
X = X.loc[indices_validos]

print(f"Tamanho depois: {X.shape}")



Iniciando limpeza profunda dos dados...
Tamanho antes: (209716, 78)
Tamanho depois: (209328, 78)

Tipos das colunas agora (Amostra):
Dst Port         int64
Protocol         int64
Flow Duration    int64
Tot Fwd Pkts     int64
Tot Bwd Pkts     int64
dtype: object
